# python - 비슷한 뉴스를 선정하는 기법

1. 문제상황
   컴퓨터는 문자를 논리적으로 이해하지는 못함
   
2. 코드의 논리 
   (문자 - 숫자 변경해 이해하는 컴퓨터적 사고인데) 숫자로 유사하다는 이해를 어떻게 컴퓨터에게 지시할까?
   


## 문자를 vector로 one hot encoding =bag of words vector
- 하나의 단어를 vector의 index로 인식시키는 방법


rome = [1,0,0,0,0,0,0,0,0]
paris = [0,1,0,0,0,0,0,0,0]
rome, partis = [1,1,0,0,0,0,0,0,0,0]

## 유클리드 distance

좌표상 거리 차이로 유사성 판단

## cosine distance
cosine similarity 로 유사성 판단

love, hate 코사인 simiarilty 사용

# data set

> 스포츠 기사 데이터 

> 1234 야구 5678축구

## process

- 파일 불러오기
- 파일 읽어서 단어사전 corpus 만들기
> 신문 1개 별로 split 하고 , 소문자로 일괄변경, set으로 묶음_단어 종류가 궁금할 것이기 때문 
- 단어별로 index만들기
- 만들어진 인덱스로 문서별로 bag of words vector 생성
> data가 작을 때도 유용함
- 비교하고자 하는 문서 비교
> 여기서는 cosine similarity 사용 문서 1개와 다른 것들을 비교
- 얼마나 맞는지 측정
> 정확도 측정도 중요합니다.


# 코드 분할 필기

### 데이터 읽어오기

In [2]:
import os

def get_file_list(dir_name):
    return os.listdir(dir_name)

if __name__ == "__main__" :
    dir_name="news_data"
    file_list = get_file_list(dir_name)
    # 폴더 속 파일명들 가져와서 list화
    file_list = [os.path.join(dir_name,file_name) for file_name in file_list]
    # python에서는 주소를 가져올때 join을 쓰는데 흔히 쓰는 파일 합치기의 기능과 더불어 os에 자동
    # 적으로 join을 적용한다는 장점 
    #print(file_list) #파일과 폴더명이 합쳐져서 나온다.
    
    x_text, y_class = get_gontents(file_list)
    
    corpus = get_corpus_dict(x_text)
    # 80개 문서의 모두 단어의 갯수만큼 벡터가 발생합니다.
    # 지금 이 분석에서는 데이터 is are am 등 을 처리하지 않았습니다.

80


### 파일별로 내용읽기

In [ ]:
def get_contents(file_list):
    y_class = []
    x_text = []
    class_dict = {
        1:"0",2:'0',3:"0",4:"0", 5:'1',6:'1',7:'1',8:'1'}
    
    for file_name in file_list:
        try:
            f = open(file_name,"r",encoding ="cp949")
            # windows 인코딩 cp949
            category = int(file_name.split(os.sep)[1].split("_")[0])
            # 아까 위에서 path로 합쳤기 때문에 os.sep기준으로 split하는 것이야
            y_class.append(class_dict[category])
            x_text.append(f.read())
            # x_text 기사 각각의 내용을 다 합쳐두었습니다.
            f.close()
            except UnicodeDecodeError as e:
                print(e)
                print(file_name)
    return x_text,y_class

### corpus 만드기 +단어별 index 생성하기 의미없는 문장부호 제거하기

In [ ]:
def get_cleaned_text(text): 
    import re
    text = re.sub('\W+',"",text.lower())
    return text
    # 소문자 정렬 후. re.sub는 측수부호 제거
    # 이 작업은 꼭 거치면 좋은게 love와 love. 이 같은 단어라는 인식을 시켜야하기 때문이다.
    # 좀더 심화일때는 is are am 을 동일하게 여기는 전처리 기술도 익혀야 겟다고 생각
    
def get_corpus_dict(text):
    text = [sentence.split() for sentence in text]
    clenad_words = [get_cleaned_text(word) for words in text for word in words]
    
    # text(파일 제목을 받으면) split해주고 text에 다시 list로 넣어줌
    # 뒤의 이중 for 가 빡신데 text에서 단어들을 뽑고 word다시 한글자씩 뽑아준다.
    # 그걸 모아둔게 clenad_words
    
    
    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i,v in enumerate(set(clenad_words)):
        corpus_dict[v] = i
        return corpus_dict
    
    
    
    
    
    
    
    
    
    

## 문서별로 bag of words vector 생성

In [ ]:
def get_count_vector(text,corpus):
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words]
                        for words in text]
    x_vector =[[0 for _ in range(len(corpus))] for x in range(len(text))]
    # 무식하게 0으로된 리스트 생성 3500이면 0이 3500개
    
    for i,text in enumerate(word_number_list):
        for word_number in text:
            x_vector[i][word_number] +=1
    return x_vector
    # 위에 설정해둔 큰 vector에 word 나온 만큼 해당 index에 1씩 더해준다.
    
    
    

## 비교하기

In [ ]:
import math
def get_cosine_similariy(x_vector,source):
    source_vector =x_vector[source]
    similarity_list =[]
    for target_vector in x_vector:
        similarity_list.append(get_cosine_similarity(source_vector, target_vector))
        
    return similarity_list

def get_top_n_similarity_news(similarity_score,n):
    import operator
    x ={i:v for i,v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(),key=operator.itemgetter(1))
    
    return list(reversed(sorted_x))[1,n+1]

In [ ]:
import os


def get_file_list(dir_name):
    return os.listdir(dir_name)

def get_conetents(file_list):
    y_class = []
    X_text = []
    class_dict = {
        1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"}

    for file_name in file_list:
        try:
            f = open(file_name, "r",  encoding="cp949")
            category = int(file_name.split(os.sep)[1].split("_")[0])
            y_class.append(class_dict[category])
            X_text.append(f.read())
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class



def get_cleaned_text(text):
    import re
    text = re.sub('\W+','', text.lower() )
    return text


def get_corpus_dict(text):
    text = [sentence.split() for sentence in text]
    clenad_words = [get_cleaned_text(word) for words in text for word in words]

    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i, v in enumerate(set(clenad_words)):
        corpus_dict[v] = i
    return corpus_dict


def get_count_vector(text, corpus):
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words] for words in text]
    X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))]

    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
    return X_vector

import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list


def get_top_n_similarity_news(similarity_score, n):
    import operator
    x = {i:v for i, v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))

    return list(reversed(sorted_x))[1:n+1]

def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]

    return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)


if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]

    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)
    source_number = 10

    result = []

    for i in range(80):
        source_number = i

        similarity_score = get_similarity_score(X_vector, source_number)
        similarity_news = get_top_n_similarity_news(similarity_score, 10)
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(sum(result) / 80)

# 통합 완성 코드

In [1]:
# 1. 데이터 읽어오기 -----------------------------------------------
import os


def get_file_list(dir_name):
    return os.listdir(dir_name)

# 2. 파일별로 내용 읽어오기 -----------------------------------------
def get_conetents(file_list):
    y_class = []
    X_text = []
    class_dict = {
        1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"}

    for file_name in file_list:
        try:
            f = open(file_name, "r",  encoding="cp949")
            # windows 인코딩 cp949, open(파일명, 읽기상태, 인코딩)
            category = int(file_name.split(os.sep)[1].split("_")[0])
            # 아까 위에서 os path로 합쳤기 때문에 os.sep 기준으로 split하는 것임
            y_class.append(class_dict[category])
            X_text.append(f.read())
            # x_text에는 기사의 내용이 다 합쳐져 있습니다.
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class


# 3. 전처리 강화
def get_cleaned_text(text):
    import re
    text = re.sub('\W+','', text.lower() )
    return text
    # 소문자 정렬 후 re.sub는 촉수부호 제거
    # 이 작업은 꼭 거치면 좋은게 love와 love. 같은 단어를 다른 단어로 인식 시키는 것을 막기 위함
    # 좀 더 심화는 is are am같은 것을 제거해주어야함.

# 4. corpus 만들기 + 단어별 index 생성하기 
def get_corpus_dict(text):
    text = [sentence.split() for sentence in text]
    clenad_words = [get_cleaned_text(word) for words in text for word in words]
    
    # text(파일 제목을 받으면) split 해주고 text에 다시 list로 넣어줌
    # 뒤의 이중 for 가 빡센데 text에서 단어들을 뽑고: word, 다시 한글자씩 뽑아준다.
    # 그걸 모아둔게 clenad_words
    
    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i, v in enumerate(set(clenad_words)):
        corpus_dict[v] = i
    return corpus_dict


def get_count_vector(text, corpus):
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words] for words in text]
    X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))]
    # 무식하게 0으로된 리스트 생성 3500이면 0이 3500개 (아주 불편한 수법이지만 )
    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
            
    # enumerate 수법으로 한번에 그 위치에 값을 증가시켰다.
    return X_vector

import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list


def get_top_n_similarity_news(similarity_score, n):
    import operator
    x = {i:v for i, v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))

    return list(reversed(sorted_x))[1:n+1]

def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]

    return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)


# -------------- 실행되는 메인 파일 -----------------
if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    # 폴더 속 파일명들을 가져와서 list화 시킨다. 
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]
    # python에서는 주소를 가져올 때 join을 쓰는데 흔히 쓰는 파일 합치기의 기능과 더불어 os종류에 관계없이
    # 자동적으로 join을 적용
    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)
    source_number = 10

    result = []

    for i in range(80):
        source_number = i

        similarity_score = get_similarity_score(X_vector, source_number)
        similarity_news = get_top_n_similarity_news(similarity_score, 10)
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(sum(result) / 80)

Number of words : 4024
0.6950000000000001


# scikit- learn을 이용해서 단어를 뽑아서 벡터화 하여 유사성 비교
> 위의 방법은 너무 구리며 실전에서는 이렇게 사용합니다.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

corpus = ["this is black cow", "this is white cow","this is red cow"]

X = vectorizer.fit_transform(corpus)
print(X.toarray())

vectorizer.get_feature_names()


[[1 1 1 0 1 0]
 [0 1 1 0 1 1]
 [0 1 1 1 1 0]]


['black', 'cow', 'is', 'red', 'this', 'white']

In [ ]:
if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    # 폴더 속 파일명들을 가져와서 list화 시킨다. 
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]
    # python에서는 주소를 가져올 때 join을 쓰는데 흔히 쓰는 파일 합치기의 기능과 더불어 os종류에 관계없이
    # 자동적으로 join을 적용
    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    
    
    